# Grenzen des Maschinellen Lernens

Das Maschinelle Lernen stößt an bestimmten Stellen an seine Grenzen.
Für die meisten Business-Cases wird mithilfe Maschinellen Lernens nicht-lineare Zusammenhänge in strukturierten Datensätzen gesucht.
Diese Zusammenhänge liegen in der Praxis aber gar nicht zwangsläufig vor.
Dies hält aber manche Praktiker nicht davon ab, die Auswertung der eigenen Daten immer wieder zu verändert, bis auf einmal das Modell zu funktionieren scheint.
Dies wird P-Hacking genannt (der Begriff stammt eigentlich aus der Statistik und hatte ursprünglich eine leicht andere Bedeutung).
Der Begriff bedeutet, dass das lang erwünschte Ergebnis leider rein zufällig zustande kam.
Dies tritt vor allem bei kleinen Datensätzen auf.

Um besonders gute Genauigkeitswerte für Modell des Maschinellen Lernens zu erhalten, können z. B. Scheinkorrelationen (d.h. lineare zufällige Zusammenhänge) oder andere nicht-lineare zufällige Zusammenhänge "ausgenutzt" werden.
Wenn genügend Attribute gleichzeitig betrachtet werden, gibt es mit hoher Wahrscheinlichkeit ein Attribut, welches einen Zusammenhang mit der Zielvariable aufweist.
Deswegen sollte immer sehr kritisch betrachtet werden, aus welchen Daten welches Ergebnis angeblich vorhersagbar sein soll.

So ein Verhalten ist ein Problem für alle Bereiche:
In der Wissenschaft werden unter Umständen falsche Annahmen und Vorhersagemodelle verwendet.
In der Praxis bedeutet es, dass ggf. defekte Vorhersagemodelle in den Betrieb aufgenommen werden.
Sobald sich jemand auf die Vorhersagen des falschen Modells verlässt, kann dies zu ernsthaften Schäden an Menschen oder Umwelt führen.
Ebenso sind finanzielle Schäden nicht ausgeschlossen.

In [ ]:
import random
import pandas as pd
import sklearn.tree

Es wird zunächst ein Datensatz mit zufälligen Attribute erzeugt.
Dies könnte z. B. die Zahlenrepräsentation von nominalskalierten Attributen sein.

In [ ]:
# Fixiere Zufallsgenerator für random.choice
random.seed(0)

number_rows = 50

df = pd.DataFrame({
    "Outlook": [random.choice(["Sunny", "Overcast", "Rain"]) for _ in range(number_rows)],
    "Temperature": [random.choice(["Hot", "Mild", "Cool"]) for _ in range(number_rows)],
    "Humidity": [random.choice(["High", "Normal"]) for _ in range(number_rows)],
    "Wind": [random.choice(["Weak", "Strong"]) for _ in range(number_rows)],
    "Play Tennis?": [random.choice(["Yes", "No"]) for _ in range(number_rows)],
})

df

In [ ]:
df_cat = df.assign(**{
    col: df[col].astype('category').cat.codes for col in ["Humidity", "Wind", "Play Tennis?"]
})
df_cat = df_cat.assign(
    Outlook=df["Outlook"].astype(
        pd.CategoricalDtype(categories=["Rain", "Overcast", "Sunny"], ordered=True)).cat.codes,
    Temperature=df["Temperature"].astype(
        pd.CategoricalDtype(categories=["Cool", "Mild", "Hot"], ordered=True)).cat.codes
)
df_cat

Nun lass uns betrachten, ob der Entscheidungsbaum einen zufälligen Zusammenhang findet, und das Kreuzvalidierungsergebnis dennoch gut aussieht.

In [ ]:
dt = sklearn.tree.DecisionTreeClassifier(random_state=0)
eingabe = df_cat[list(set(df_cat.columns) - set(["Play Tennis?"]))]

ziel = df["Play Tennis?"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(eingabe, ziel, random_state=42)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

<span style="color:blue; font-weight:bold">Aufgabe 1</span>

Interpretieren Sie den Zahlenwert.
Liegt hier ein zufälliger nicht-linearer Zusammenhang vor?
Ist dieses Ergebnis plausibel?
Warum (nicht)?

*Antwort*: ...

## Feature Engineering falsch gemacht

Normalerweise hilft Feature Engineering dabei, zielführende Zahlenwerte zu generieren.
So ist es z. B. sinnvoll, statt einem Start- und einem Endzeitpunkt lieber gleich die Zeitspanne zu generieren, oder statt einer Länge und einer Breite gleich die Fläche zu berechnen.
Es gibt aber auch Fälle, in denen Projektverantwortliche beliebige Attribute addiert, multipliziert etc. haben, ohne dass es dafür eine inhaltliche Rechtfertigung gab.
Genau das werden wir jetzt auf die Spitze treiben.

Nun werden wir so lange neue Attribute erstellen, bis wir (mindestens) eines finden, mit dem der Entscheidungsbaum zu einem guten Ergebnis kommt.

In [ ]:
new_columns = {}

for column_A in list(set(df_cat.columns) - set(["Play Tennis?"])):
    for column_B in list(set(df_cat.columns) - set(["Play Tennis?"])):
        if column_A == column_B:
            continue

        addition = df_cat[column_A] + df_cat[column_B]
        addition.name = f"{column_A} + {column_B}"
        new_columns.update({addition.name: addition})

        subtraktion = df_cat[column_A] - df_cat[column_B]
        subtraktion.name = f"{column_A} - {column_B}"
        new_columns.update({subtraktion.name: subtraktion})

        multiplication = df_cat[column_A] * df_cat[column_B]
        multiplication.name = f"{column_A} * {column_B}"
        new_columns.update({multiplication.name: multiplication})

Diese kreierten Attribute fügen wir nun dem DataFrame hinzu.

In [ ]:
df_extended = df_cat.assign(**new_columns)
df_extended

Nun lass uns sehen, ob der Entscheidungsbaum mit diesen generierten Features besser arbeiten kann:

In [ ]:
ziel = df_extended["Play Tennis?"]
eingabe = df_extended[list(sorted(list(set(df_extended.columns) - set(["Play Tennis?"]))))]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(eingabe, ziel, random_state=74)
dt = sklearn.tree.DecisionTreeClassifier(random_state=5)
dt.fit(X_train, y_train)
s = dt.score(X_test, y_test)
s

<span style="color:blue; font-weight:bold">Aufgabe 2</span>

Interpretieren Sie den Zahlenwert.
Liegt hier ein zufälliger nicht-linearer Zusammenhang vor?
Ist dieses Ergebnis plausibel?
Warum (nicht)?

*Antwort*: ...

<span style="color:blue; font-weight:bold">Aufgabe 3</span>

Stellen Sie sich vor, Sie haben mit einem Dienstleister einen Vertrag abgeschlossen.
Dort heißt es, es soll mindestens eine Genauigkeit von 80% erreicht werden (wie oben der Fall).
Wie können Sie sich davor schützen, dass der Dienstleister Ihnen ein nicht praxistaugliches Produkt verkauft?
Gehen Sie davon aus, dass Sie den Source Code nicht einsehen können.

*Antwort*: ...

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.